# 18.模仿学习（Imitation Learning，简称IL）
> 虽然 **强化学习** 不需要有监督学习中的标签数据，但它十分依赖 **奖励函数** 的设置。有时在 **奖励函数** 上做一些微小的改动，训练出来的策略就会有天差地别。而且在现实场景中， **奖励信号** 总是难以明确：
- 例如，对于无人驾驶车辆智能体的规控，其观测是当前的环境感知恢复的 3D 局部环境，动作是车辆接下来数秒的具体路径规划，那么奖励是什么？如果只是规定正常行驶而不发生碰撞的奖励为+1，发生碰撞为-100，那么智能体学习的结果则很可能是找个地方停滞不前。**奖励函数** 往往需要精心设计和调试。
> 那有什么方法可以跳过 **奖励函数** 的设计呢？

> **模仿学习** 研究的便是这一类问题，在模仿学习的框架下，存在 **专家智能体**，其在某个环境下提供的一系列 **状态动作对**，被认为是 **最优行为策略**。**模仿者**的任务则是利用这些 **专家数据** 进行训练，**无须奖励信号** 就可以达到一个接近专家的策略。
> 学术界关于 **模仿学习** 的传统核心方法分为 **3 类**：

| 方法                                                            | 概念                                                                                           | 缺点                      | 适用任务                                 | 最早时间 | 论文链接                                                                                                     |
|---------------------------------------------------------------|----------------------------------------------------------------------------------------------|-------------------------|--------------------------------------|------|----------------------------------------------------------------------------------------------------------|
| **行为克隆（behavior cloning，BC）**                                 | 通过监督学习模仿专家的状态-动作对，直接复制专家的行为                                                                  | - 缺乏长期奖励优化<br> - 数据依赖性强 | 数据充足且专家行为完美的任务，如机器人抓取、路径规划等          | 1988 | [论文学习](https://papers.neurips.cc/paper_files/paper/1988/file/812b4ba287f5ee0bc9d43bbf5bbe87fb-Paper.pdf) |
| **逆强化学习（inverse RL）**                                         | 不同于行为克隆仅仅是复制专家的动作，它的目标是从专家的行为中推断出奖励函数。通过推断奖励函数，智能体就可以基于奖励来优化自己的策略了                           | - 计算复杂度高<br> - 数据需求大    | 无法直接获取奖励信号或奖励函数难以设计的任务，如自主驾驶、复杂策略规划。 | 2000 | [论文学习](https://www.cl.cam.ac.uk/~ey204/teaching/ACS/R244_2022_2023/papers/NG_ICML_2000.pdf)              |
| **生成式对抗模仿学习（generative adversarial imitation learning，GAIL）** | 结合生成对抗网络（GANs）将模仿学习视为一个对抗过程，其中一个生成器（模仿者）试图模仿专家的行为，另一个判别器（判别器）则尝试判断动作是否来自专家。最终，生成器学会如何模仿专家的策略 | - 训练不稳定<br> - 对样本质量敏感   | 没有明确奖励信号的复杂任务，如机器人运动、视频游戏AI等。        | 2016 | [论文学习](https://arxiv.org/abs/1606.03476)                                                                 |


> 本节主要介绍 **行为克隆** 和 **生成式对抗模仿学习**
> **逆强化学习** 有良好的学术贡献，但由于其计算复杂度较高，实际应用的价值较小

## 18.1 行为克隆
> **行为克隆（BC）** 直接使用 **监督学习** 方法：
$$\theta^*=\arg\min_\theta\mathbb{E}_{(s,a)\sim B}[\mathcal{L}(\pi_\theta(s),a)]$$
- $B$是专家的数据集，$\mathcal{L}$是对应监督学习框架下的损失函数
- 若动作是离散的，该损失函数可以是最大似然估计得到的；若动作是连续的，该损失函数可以是均方误差函数
> 但也造成了其存在很大的局限性：由于训练使用的 **专家数据** 难以囊括所有状态分布，因此 BC 只能在专家数据的状态分布下预测得比较准。而且，**强化学习** 面对的是一个序贯决策问题，，只要存在一点偏差，就有可能导致下一个遇到的状态是在专家数据中没有见过的。此时，由于没有在此状态（或者比较相近的状态）下训练过，策略可能就会随机选择一个动作，这会导致下一个状态 **进一步偏离** 专家策略遇到的的数据分布。最终，该策略在真实环境下不能得到比较好的效果，这被称为行为克隆的 **复合误差（compounding error）** 问题：
![行为克隆带来的复合误差问题](Illustrations/行为克隆带来的复合误差问题.png)
> 即使 **行为克隆** 有这些局限，但其实现十分简单，因此在很多实际场景下它都可以作为 **策略预训练** 的方法：使 **策略** 无须在 **最开始低效地** 通过和环境交互来探索较好的动作，而是通过模仿专家智能体的行为数据来 **快速达到较高水平**，为接下来的强化学习创造一个 **高起点**。

## 18.2 生成式对抗模仿学习
> **生成式对抗模仿学习** 是2016年由斯坦福大学研究团队提出的 **基于生成式对抗网络** 的**模仿学习**，它诠释了**生成式对抗网络**的 **本质** 其实就是**模仿学习**
> **GAIL** 算法中有一个 **判别器** 和一个 **策略**：
- **策略** 就相当于是生成式对抗网络中的 **生成器（generator）**，给定一个状态，**策略** 会输出这个状态下应该采取的动作
- **判别器（discriminator）$D_\phi$** 将状态动作对$(s,a)$作为输入，输出一个 0 到 1 之间的实数，表示 **判别器** 认为该状态动作对来自 **智能体策略$\theta$** 而非 **专家$E$** 的概率。**判别器** 的目标是尽量将专家数据的输出靠近 0，将模仿者策略的输出靠近 1，这样就可以 **将两组数据分辨开**。

> 于是**判别器$D_\phi$** 的目标是最大化对数似然：
$$\mathcal{L}_D(\phi)=\mathbb{E}_{(s,a)\sim\pi_\theta}[\log D_\phi(s,a)]+\mathbb{E}_{(s,a)\sim\pi_E}[\log(1-D_\phi(s,a))]$$

> 实际训练时，取负，最小化损失函数（本质上为 **二分类交叉熵损失**，依据**真实采样分布**计算与**预测分布**的差异）：
$$\min_\phi-\mathcal{L}_D(\phi)=-\mathbb{E}_{(s,a)\sim\pi_\theta}[\log D_\phi(s,a)]-\mathbb{E}_{(s,a)\sim\pi_E}[\log(1-D_\phi(s,a))]$$

> 有了 **判别器$D_\phi$** 之后，**策略** 的目标就是其交互产生的轨迹能被 **判别器** 误认为专家轨迹，所以将 **判别器的输出** 作为 **奖励函数** 来训练 **模拟者的策略**：
$$r(s,a)=-\log D(s,a)$$

- **在不断的对抗过程中**：训练初期，**判别器**能够轻易区分专家数据和策略数据，因而对专家样本输出接近 0、对策略样本输出接近 1；随着**策略**不断更新以获得更高奖励，它会逐渐生成更接近专家的数据分布，使得**判别器**的区分难度增大，输出趋于模糊；在理论上的收敛点（**纳什均衡**）时，**策略分布与专家分布**完全一致，**判别器**无法再区分二者，只能对所有样本给出约 0.5 的概率，从而体现出博弈的平衡。

> 可见，**GAIL** 中的 **策略** 需要和环境进行交互，这一点和 **BC** 不同，**BC** 完全不需要和环境交互
> **GAIL** 实质上是模仿了 **专家策略的占用度量**，即 **尽量使得** 在环境中 **策略的占用度量$\rho_{\pi}(s,a)$** 和 **专家策略的占用度量$\rho_{E}(s,a)$** 一致：
![GAIL的优化目标](Illustrations/GAIL的优化目标.png)

## 18.3 代码实践（CartPole-v1）
- 生成专家数据
- **行为克隆** 算法实现
- **生成式对抗模仿学习** 算法实现

导入相关库：

In [4]:
# 基本库
import numpy as np
import copy

from utils.smoothing import moving_average
from utils.advantage import compute_advantage
from utils.training import train_on_policy_agent
# 神经网络
import torch
import torch.nn.functional as F
# Gymnasium 是一个用于开发和测试强化学习算法的工具库，为 OpenAI Gym 的更新版本（2021迁移开发）
import gymnasium as gym

### 生成专家数据：
> 通过 **PPO** 算法训练出一个表现良好的专家模型，再利用专家模型生成专家数据

### 行为克隆的代码实践:

### 生成式对抗模仿学习的代码实践: